## What the important is not the arrival point, it is the journey itself. Enjoy your journey :)

# Including 
* Single-Particle Theta and Real Space Hopping Phases
* Essential First Band Approximation Functions

In [17]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Theta Space.ipynb")
@nbinclude(joinpath("..","First Band Approximation Functions.ipynb"));

# Initial Parameters

In [18]:
# Lattice Size
Nx=3; Ny=3; N=Nx*Ny

# Magnetic Flux Per Plaquet
p=1; q=Ny; alpha=p/q

# Total Particle Number
# NON-INTERACTING CASE:
# PN = 2

# INTERACTING CASE
PN = [0, 1]

# Cut-Off for Sub-Space (Real Space for now...)
cut_off = Nx*Ny

# Interaction Potential
U = 0

# Theta Parameter Space
T_size = 5;

# Single Particle Operator

In [19]:
function Sp_Theta_Op(Nx, Ny, alpha, sp_basis, Tx, Ty)
        
    H_T = HSP_T(Nx, Ny, alpha, Tx, Ty, 0)
    
    H = SparseOperator(sp_basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

Sp_Theta_Op (generic function with 1 method)

In [20]:
using LinearAlgebra

# Reference Parameters
Tx = Ty = 0
sp_basis = NLevelBasis(N)
# eigenenergies(dense(Sp_Theta_Op(Nx, Ny, alpha, sp_basis, Tx, Ty))) == eigvals(HSP_T(Nx, Ny, p/q, Tx, Ty, 0))
# eigvals(HSP_T(Nx, Ny, p/q, Tx, Ty, 0))
# eigen(HSP_T(Nx, Ny, p/q, Tx, Ty, 0))

NLevel(N=9)

# Sub-Space Many-Body

In [21]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme");

In [22]:
basis_cut_mb, basis_cut_sp = get_Bosonic_MB_Basis(cut_off,PN);

In [23]:
function get_total_H(Nx, Ny, alpha, sp_basis, Tx, Ty, cut_off, PN, basis_cut_sp, basis_cut_mb)
    # Single Particle Operator
    H1 = Sp_Theta_Op(Nx, Ny, alpha, sp_basis, Tx, Ty)
    
    # Sub-States Go on to Cut-Off
    sub_states = get_sub_states(H1, cut_off)

    # Sub-Space Basis and Projection Operator
    basis_sub, P, Pt = get_projector_op(sub_states, sp_basis)

    # Single-Particle Sub-Space Operator
    H1_sub = get_subspace_op(H1, P, Pt)

    # Many-Body Sub-Space Bosonic Fock States
    states_mb = bosonstates(basis_sub, PN)     

    # Many-Body Sub-Space Basis
    basis_mb = ManyBodyBasis(basis_sub, states_mb)

    # Kinetic Term of Many-Body Sub-Space Operator
    H1_MB = get_mb_op(basis_mb, basis_sub, H1_sub)
    H1cut = SparseOperator(basis_cut_mb)
    H1cut.data = H1_MB.data

    # Interaction Term of Many-Body Sub-Space Operator
    H_Int = Hubbard_Interaction(basis_cut_sp, basis_cut_mb, P, Pt, cut_off)
    
    return H1cut + H_Int
end

get_total_H (generic function with 1 method)

In [24]:
# PN

In [25]:
# Tx = Ty = 0
# H_total = get_total_H(Nx, Ny, alpha, sp_basis, Tx, Ty, cut_off, PN, basis_cut_sp, basis_cut_mb);
# EE, VV = eigen(dense((H_total+H_total')/2).data)
# VV[:,4]

# Exact Diagonalization

In [26]:
# using DataFrames

# function get_filtered_energies(pn, basis, H_total)
    
#     # Operator Form of Eigenstates
#     E, V = eigenstates(dense(dense((H_total+H_total')/2)))

#     # Energies According to Each Particle Number
#     PN_Energies = Array{Float64}(undef, length(E), 2)
#     for i in 1:length(E)
#         # Particle Number in Each State
#         PN_Energies[i] = round(expect(number(basis), V[i]))
        
#         # Energies in Each State
#         PN_Energies[i,2] = E[i]
#     end
    
#     # Fiter Process
#     Filtered_Energies = DataFrame(PN_Energies, :auto)
#     Filtered_Energies = filter(row -> (row.x1 == pn),  Filtered_Energies)
    
#     return Filtered_Energies
# end

In [27]:
# # Energies of Filtered Particle Number 
# pn = 1
# df = get_filtered_energies(pn, basis_cut_mb, H_total)
# filtered_energies = Matrix(df)[:,2]

In [28]:
# using Plots

# # Plots Bands
# scatter(1:length(filtered_energies), filtered_energies, legend=false, axis=nothing)

In [29]:
# # Eigen-States of Filtered Particle Number 
# function Filtered_Hubbard_States(filtered_energies, states)
#     number_of_states = length(filtered_energies)
#     return states[:,1:number_of_states]
# end

In [30]:
# pn = 1

# function get_eigenstates(basis_cut_mb, H_total, pn)
    
#     E, V = eigen(dense((H_total+H_total')/2).data)

#     # Sorting Again...
#     #V = V[:, sortperm(E)]
    
#     df = get_filtered_energies(pn, basis_cut_mb, H_total)
    
#     filtered_energies = Matrix(df)[:,2]
    
#     #filtered_states = Filtered_Hubbard_States(filtered_energies, V)
#     #filtered_states = dropcol(V, 4)
    
#     return filtered_energies, V
# end

# # Orthogonality 
# # filtered_energies, filtered_states = get_eigenstates(basis_cut_mb, H_total, pn)
# # for i in 1:N
# #     j = 9
# #     a = filtered_states[:,j]
# #     b = filtered_states[:,i]
# #     println(dot(a,b))
# # end
# Tx = Ty = 0
# H_total = get_total_H(Nx, Ny, alpha, sp_basis, Tx, Ty, cut_off, PN, basis_cut_sp, basis_cut_mb)
# get_eigenstates(basis_cut_mb, H_total, pn)[1]

# CHERN CALCULATION SECTION

In [56]:
function findcol(M, col)                
    @inbounds @views for c in axes(M, 2)
       M[:,c] == col && return c       
    end                                 
    return nothing                      
end
col = [
    1.0 + 0.0im;
 0.0 + 0.0im;
 0.0 + 0.0im;
 0.0 + 0.0im;
 0.0 + 0.0im;
 0.0 + 0.0im;
 0.0 + 0.0im;
 0.0 + 0.0im;
 0.0 + 0.0im;
 0.0 + 0.0im
]
dropcol(M::AbstractMatrix, j) = M[:, deleteat!(collect(axes(M, 2)), j)];

In [32]:
# Mesh Grid for Theta Space Parameter
dx=2*pi/T_size; dy=dx;
Tx=range(start=0, stop=2*pi-dx, step=dx)
Ty=range(start=0, stop=2*pi-dy, step=dy)

0.0:1.2566370614359172:5.026548245743669

In [135]:
function Chern_Nums(n1, n2)
    
    Sum=0
    
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            
            H_Total = get_total_H(Nx, Ny, alpha, sp_basis, Tx[tx], Ty[ty], cut_off, PN, basis_cut_sp, basis_cut_mb)
            
            w1, v1 = eigen(dense((H_Total+H_Total')/2).data)
            
            # sorting eigenvalues
            i = sortperm(w1, by=real);w1 = w1[i]
            # sorting eigenstates
            v1 = v1[:,i]
            
            # if isapprox(v1[1,1], 0+0im, atol=0.1) == false
            #     v1 = v1 ./ v1[1,1]
            #     v1 = v1/norm(v1)
            # end

            drop_idx = findcol(v1, col)
            v1 = dropcol(v1, drop_idx)
            
            # multiplet eigenstates
            v1 = v1[:,n1:n2]
            
            #------------------------------------
            H_Total = get_total_H(Nx, Ny, alpha, sp_basis, Tx[tx]+dx, Ty[ty], cut_off, PN, basis_cut_sp, basis_cut_mb)
            
            w2, v2 = eigen(dense((H_Total+H_Total')/2).data)
            
            i = sortperm(w2, by=real);w2 = w2[i]
            v2 = v2[:,i]

            # if isapprox(v2[1,1], 0+0im, atol=0.1) == false
            #     v2 = v2 ./ v2[1,1]
            #     v2 = v2/norm(v2)
            # end

            drop_idx = findcol(v2, col)
            v2 = dropcol(v2, drop_idx)
            
            v2 = v2[:,n1:n2]
            
            #------------------------------------
            H_Total = get_total_H(Nx, Ny, alpha, sp_basis, Tx[tx], Ty[ty]+dy, cut_off, PN, basis_cut_sp, basis_cut_mb)
            
            w3, v3 = eigen(dense((H_Total+H_Total')/2).data)
            
            i = sortperm(w3, by=real);w3 = w3[i]
            v3 = v3[:,i]

            # if isapprox(v3[1,1], 0+0im, atol=0.1) == false
            #     v3 = v3 ./ v3[1,1]
            #     v3 = v3/norm(v3)
            # end

            drop_idx = findcol(v3, col)
            v3 = dropcol(v3, drop_idx)
            
            v3 = v3[:,n1:n2]
            
            #------------------------------------
            H_Total = get_total_H(Nx, Ny, alpha, sp_basis, Tx[tx]+dx, Ty[ty]+dy, cut_off, PN, basis_cut_sp, basis_cut_mb)
            
            w4, v4 = eigen(dense((H_Total+H_Total')/2).data)
            
            i = sortperm(w4, by=real);w4 = w4[i]
            v4 = v4[:,i]

            # if isapprox(v4[1,1], 0+0im, atol=0.1) == false
            #     v4 = v4 ./ v4[1,1]
            #     v4 = v4/norm(v4)
            # end

            drop_idx = findcol(v4, col)
            v4 = dropcol(v4, drop_idx)
            
            v4 = v4[:,n1:n2]
            
            #----------LINK VARIABLES------------
            U1=det(adjoint(v1)*v2)
            U1=U1/abs(U1)
            U2=det(adjoint(v2)*v4)
            U2=U2/abs(U2)
            U3=det(adjoint(v3)*v4)
            U3=U3/abs(U3)
            U4=det(adjoint(v1)*v3)
            U4=U4/abs(U4)
            
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
            
        end
    end
    
    return 1/(2*pi*1im)*Sum
end

Chern_Nums (generic function with 1 method)

In [136]:
Chern_Nums(1, 3)

-1.1089746373955864e-16 + 3.5339496460705774e-17im

# Check with "Hofstadter MB Chern" file

In [132]:
pn = 1
tx = 2
ty = 2
#---
H_Total = get_total_H(Nx, Ny, alpha, sp_basis, Tx[tx], Ty[ty], cut_off, PN, basis_cut_sp, basis_cut_mb)     
w1, v1 = eigen(dense((H_Total+H_Total')/2).data)
drop_idx = findcol(v1, col)
v1 = dropcol(v1, drop_idx)
#---
H_Total = get_total_H(Nx, Ny, alpha, sp_basis, Tx[tx]+dx, Ty[ty], cut_off, PN, basis_cut_sp, basis_cut_mb)     
w2, v2 = eigen(dense((H_Total+H_Total')/2).data)
drop_idx = findcol(v2, col)
v2 = dropcol(v2, drop_idx)
#---
H_Total = get_total_H(Nx, Ny, alpha, sp_basis, Tx[tx], Ty[ty]+dy, cut_off, PN, basis_cut_sp, basis_cut_mb)     
w3, v3 = eigen(dense((H_Total+H_Total')/2).data)
drop_idx = findcol(v3, col)
v3 = dropcol(v3, drop_idx)
#---
H_Total = get_total_H(Nx, Ny, alpha, sp_basis, Tx[tx]+dx, Ty[ty]+dy, cut_off, PN, basis_cut_sp, basis_cut_mb)     
w4, v4 = eigen(dense((H_Total+H_Total')/2).data)
drop_idx = findcol(v4, col)
v4 = dropcol(v4, drop_idx)
#---
# U1=det(adjoint(v1)*v2)
# U1=U1/abs(U1)
# println(U1)
# U2=det(adjoint(v2)*v4)
# U2=U2/abs(U2)
# println(U2)
# U3=det(adjoint(v3)*v4)
# U3=U3/abs(U3)
# println(U3)
# U4=det(adjoint(v1)*v3)
# U4=U4/abs(U4)
# println(U4)

10×9 Matrix{ComplexF64}:
          0.0+0.0im          …           0.0+0.0im
    -0.272886-0.0575786im       -1.52656e-16-1.38778e-17im
   -0.0181575+0.0124418im       -1.10589e-17+7.58942e-18im
    -0.702322+0.654582im        -2.35922e-16+2.15106e-16im
 -7.56339e-16+1.40166e-15im     -1.95156e-17-4.33681e-17im
  3.83027e-15-2.87964e-16im  …  -2.67147e-16+3.46945e-18im
 -2.35922e-16+1.49186e-16im      5.63785e-18-1.34441e-17im
  2.77556e-16+3.29597e-17im       -0.0263206-0.00798375im
  1.11022e-16+1.11022e-16im       -0.0552838-0.0259299im
 -3.18827e-16-0.0im                 0.997755+0.0im

In [133]:
w1

10-element Vector{Float64}:
 -2.5466393943271624
 -2.54663939432716
 -2.546639394327157
  0.0
  0.20750036611082442
  0.20750036611082545
  0.20750036611082567
  2.339139028216333
  2.3391390282163353
  2.3391390282163362

In [134]:
v1

10×9 Matrix{ComplexF64}:
          0.0+0.0im                   0.0+0.0im          …  0.0+0.0im
     0.142217+0.0198581im        0.337418+0.243865im        0.0+0.0im
    -0.239493+0.0865135im        0.794149+0.40476im         0.0+0.0im
      0.93695+0.191477im        0.0760329+0.162463im        0.0+0.0im
  4.70312e-16-6.18727e-17im  -5.20714e-16-2.18237e-16im     0.0+0.0im
  5.30345e-16+5.9649e-17im    2.27636e-16-2.46812e-17im  …  0.0+0.0im
   2.4873e-16+7.60546e-18im   6.12928e-16+2.74883e-16im     0.0+0.0im
 -7.86804e-18-1.11336e-16im   1.01227e-16+1.77661e-16im     0.0+0.0im
  1.70905e-17-2.13632e-18im   2.05086e-16+3.41811e-17im     0.0+0.0im
          0.0+0.0im                   0.0+0.0im             1.0+0.0im

In [137]:
U1

-0.9764095033990257 + 0.21592702858157442im

In [138]:
U2

0.8573655685056786 + 0.514707957914908im

In [139]:
U3

-0.4631871192962307 - 0.8862605105261427im

In [140]:
U4

0.7205629143460544 - 0.6933895632825176im